In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# SVR (Kernel)

## Préparation des données V1 - 2023/11/16

Stratégie :    
On supprime la colonne date et la colonne way. On remplace les NAN par des 0

- Import des données

In [2]:
path = "../../data"
x_data = pd.read_csv(path+'/Xtrain_hgcGIrA.csv', sep=',') # features
y_data = pd.read_csv(path+'/Ytrain_yL5OjS4.csv', sep=',', usecols=[1]) # occupancy rate

- Mise en forme des données

In [3]:
x_data = x_data.drop("date", axis=1) # suppression de la colonne date
x_data = x_data.drop("way", axis=1) # suppression de la colonne way

#on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype('category')
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)


x_data["hour"] = x_data["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
) # on remplace les Nan de hour par des 0 et les heures par des entiers

#on remplace les Nan des pxqx par 0
x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]].fillna(0)

- Création d'un dataset de test et d'apprentissage

In [4]:
y_data = y_data.iloc[x_data.index]
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(
    x_data, y_data, test_size=0.2
)

## SVR - 2023/11/16 - Préparation des données V1

- Import de SVR

In [5]:
from sklearn.svm import SVR

- Test sans jouer sur les paramètres

In [6]:
SVR_base = SVR()
SVR_base.fit(x_data_train.values, y_data_train.values.ravel())

print("Score train", SVR_base.score(x_data_train.values, y_data_train.values.ravel()))
print("Score test", SVR_base.score(x_data_test.values, y_data_test.values.ravel()))

Score train 0.8580803421750182
Score test 0.8577649436926735


On va maintenant essayer d'optimiser les paramètres. On n'a pas de Matrice de Graham pour l'instant, donc on n'utilisera pas de kernel precomputed.

- Choix des paramètres

In [9]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from time import time

start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

c = np.linspace(0.1,20,20)
epsilon = np.linspace(0.001,0.11,15)

tuned_parameters = {'C': c, 'epsilon': epsilon}  

SVR_params = GridSearchCV(SVR(),
                      tuned_parameters,
                      cv=my_kfold,
                      n_jobs=-1)

SVR_params.fit(x_data_train.values, y_data_train.values.ravel())
print(SVR_params.score(x_data_test.values, y_data_test.values.ravel()), SVR_params.best_params_)
print(str(time() - start)+ " sec")

KeyboardInterrupt: 

Premier test avec 
- c = [0.1, 1, 5, 10]
- epsilon = [0.1, 1, 5, 10]

On obtient :

0.8867840596377132 {'C': 10, 'epsilon': 0.1}
11.792879343032837 sec


Deuxième test avec 
- c = np.linspace(0.1,15,15)
- epsilon = np.linspace(0.01,1,15)

On obtient :

0.9495510095454961 {'C': 15.0, 'epsilon': 0.01}
326.8271632194519 sec

Troisième test avec 
- c = np.linspace(0.1,20,20)
- epsilon = np.linspace(0.001,0.11,15)

On obtient :

0.9522362454847109 {'C': 20.0, 'epsilon': 0.01657142857142857}    
1874.0161483287811 sec

On décide de s'arrêter là et de garder les paramètres de ce troisième test